In [52]:
import pandas as pd

topic_number = 20

number_of_docs = 100000


df = pd.read_csv("/mnt/nfs-scratch/ECEN_403-404/team_12/James_env/abcnews-date-text.csv",
                header = 0,
                nrows = number_of_docs)

#print(df.head())

date = df['publish_date'].tolist()
headline = df['headline_text'].tolist()

#print(headline)


In [53]:
from gensim import corpora
from collections import defaultdict




stoplist = set('for a of the and to in be'.split())

texts = [ [word for word in document.lower().split() if word not in stoplist] for document in headline]

#print(text)
frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1
        
texts = [ [token for token in text if frequency[token] > 1] for text in texts]


dictionary  = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
#print(corpus)
#print(texts)

In [54]:
from gensim import models

tfidf = models.TfidfModel(corpus)
print(tfidf)

TfidfModel(num_docs=100000, num_nnz=559593)


In [55]:
corpus_tfidf = tfidf[corpus]


##for doc in corpus_tfidf:
    #print(doc)


In [56]:
lda_model = models.ldamodel.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=topic_number)
corpus_lda = lda_model[corpus_tfidf]

In [57]:
#lda_model.print_topics(20)

In [58]:
#for doc, as_text in zip(corpus_lda, headline):
    #print(doc, as_text)

In [59]:
print("Perplexity: ", lda_model.log_perplexity(corpus))

Perplexity:  -13.669946199465166


In [60]:
coherence_model_lda = models.coherencemodel.CoherenceModel(model = lda_model,texts = texts, dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print("\nCoherence Score: ", coherence_lda)


Coherence Score:  -13.02748518382138


In [61]:
topic_buckets = [[]]
for j in range(1,topic_number):
    topic_buckets.append([])
    
#print(topic_buckets)
for i in range(0,11):
    document_topic = list(models.ldamodel.LdaModel.get_document_topics(lda_model, 
                                                                       bow=corpus[i], 
                                                                       minimum_probability=None, 
                                                                       minimum_phi_value=None, 
                                                                       per_word_topics=False))
    
    
    for topic in document_topic:
        list_topic = list(topic)
        list_topic.insert(0,i)
        #rint(list_topic)
        topic_buckets[list_topic[1]].append(list_topic)
    #print(document_topic)
    
for i in topic_buckets:
    print(i)

    

[[5, 0, 0.012500793], [7, 0, 0.37271065]]
[[5, 1, 0.2606819], [7, 1, 0.2472593]]
[[0, 2, 0.5784828], [1, 2, 0.72142965], [5, 2, 0.012500793], [6, 2, 0.21499288], [7, 2, 0.010001246]]
[[3, 3, 0.22633325], [4, 3, 0.15863232], [5, 3, 0.012500793], [7, 3, 0.010001246], [10, 3, 0.21218155]]
[[5, 4, 0.012500793], [7, 4, 0.010001246]]
[[4, 5, 0.4488415], [5, 5, 0.012500793], [7, 5, 0.010001246]]
[[2, 6, 0.17499225], [5, 6, 0.012500793], [7, 6, 0.010001246]]
[[0, 7, 0.2929126], [5, 7, 0.012500793], [7, 7, 0.010001246]]
[[5, 8, 0.012500793], [7, 8, 0.010001246], [9, 8, 0.31898034]]
[[1, 9, 0.14998913], [2, 9, 0.29125455], [5, 9, 0.012500793], [7, 9, 0.010001246], [9, 9, 0.55241567]]
[[3, 10, 0.6611643], [4, 10, 0.2710697], [5, 10, 0.012500793], [7, 10, 0.010001246]]
[[5, 11, 0.012500793], [7, 11, 0.010001246]]
[[5, 12, 0.012500793], [7, 12, 0.010001246], [8, 12, 0.5518095]]
[[2, 13, 0.1749822], [5, 13, 0.012500793], [7, 13, 0.2100089], [10, 13, 0.15883815]]
[[5, 14, 0.012500793], [7, 14, 0.0100

**This Next Cell is Used to Save the Model Only Run it if you want to erase the previous model and save the new one**


In [66]:
import os
import tempfile
##tempfile.NamedTemporaryFile(mode='w+b', 
    #buffering=None, encoding=None, newline=None, suffix=None, prefix=None, dir=None, delete=Tru

with tempfile.NamedTemporaryFile(prefix='model-', suffix='.lsi', delete=False, dir = '/mnt/nfs-scratch/ECEN_403-404/team_12/James_env') as tmp:
    lda_model.save(tmp.name)  # same for tfidf, lda, ...